In [13]:
# --- Imports ---
import base64
from IPython.display import display
from ipywidgets import FileUpload, Button, Output, VBox, Text
from openai import OpenAI
import smtplib
from email.message import EmailMessage
import re

# --- OpenAI (DashScope-compatible) Client ---
client = OpenAI(
    api_key="sk-3685593a35824214a9a06e37849916f1",  # ⚠️ Replace with your actual key
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)

# --- Email Configuration ---
GMAIL_ADDRESS = "kalathilvanaja@gmail.com"           # ⚠️ Your Gmail
GMAIL_APP_PASSWORD = "rdsp qvqe gtus gaht"    # ⚠️ App password (not regular password)
RECIPIENT_EMAIL = "sujith81159@gmail.com"        # ⚠️ Recipient email

# --- Email Function ---
def send_health_alert_email(report, product_name):
    try:
        msg = EmailMessage()
        msg['Subject'] = f"Health safety concern: {product_name}"
        msg['From'] = GMAIL_ADDRESS
        msg['To'] = RECIPIENT_EMAIL
        msg.set_content(f"""⚠️ This food product '{product_name}' may pose a health risk.

Here is the AI-generated analysis report:

{report}
""")
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(GMAIL_ADDRESS, GMAIL_APP_PASSWORD)
            smtp.send_message(msg)

        with output:
            print("📧 Email alert sent successfully.")
    except Exception as e:
        with output:
            print("❌ Failed to send email:", str(e))

# --- Analyze Image via OpenAI ---
def analyze_image(base64_image, product_name):
    response = client.chat.completions.create(
        model="qwen-vl-plus",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    },
                    {
                        "type": "text",
                        "text": f"""You are a food safety expert. Analyze this food label image for the product named "{product_name}".
Extract the ingredient list and identify any harmful, banned, or controversial substances.
Explain health risks in plain language and give a safety score from 0 to 100."""
                    }
                ]
            }
        ]
    )
    return response.choices[0].message.content

# --- Widgets and Output ---
upload_widget = FileUpload(accept='image/*', multiple=False)
product_name_widget = Text(placeholder='Enter product name...', description='Product:')
analyze_button = Button(description="Analyze Label", button_style='success')
output = Output()

# --- Main Button Handler ---
def on_analyze_clicked(_):
    output.clear_output()

    product_name = product_name_widget.value.strip()
    if not product_name:
        with output:
            print("❗ Please enter the product name.")
        return

    if not upload_widget.value:
        with output:
            print("❗ Please upload an image file first.")
        return

    try:
        file_info = upload_widget.value[0]  # ✅ For older ipywidgets
        image_bytes = file_info.get('content', b'')

        if not image_bytes:
            with output:
                print("⚠️ Uploaded file is empty.")
            return

        mime_type = file_info.get('type', '')
        if not mime_type.startswith("image/"):
            with output:
                print("❗ Not a valid image file.")
            return

        base64_img = base64.b64encode(image_bytes).decode("utf-8")

        with output:
            print(f"✅ Image for '{product_name}' uploaded successfully. Analyzing...")

        result = analyze_image(base64_img, product_name)

        with output:
            print("\n--- AI Analysis Report ---\n")
            print(result)

        # --- Check for score and send alert if below 70 ---
        score_match = re.search(r"(\d{1,3})\s*/\s*100", result)
        if score_match:
            score = int(score_match.group(1))
            if score < 70:
                with output:
                    print(f"\n⚠️ Low Safety Score Detected: {score}/100")
                send_health_alert_email(result, product_name)

    except Exception as e:
        with output:
            print("❌ Error during analysis:", str(e))

# --- Attach and Display UI ---
analyze_button.on_click(on_analyze_clicked)
display(VBox([product_name_widget, upload_widget, analyze_button, output]))
